In [1]:
import pandas as pd
import ast

In [2]:
df_user_reviews = pd.read_csv('user_reviews_cleaned.csv')
df_user_items = pd.read_csv('user_items_cleaned.csv')
df_steam_games = pd.read_csv('steam_games_cleaned.csv')


In [3]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_items, df_steam_games[['item_id', 'year', 'genres']], on='item_id', how='inner')s
df_merged = df_merged.groupby(['year', 'genres'])['hours_game'].sum().reset_index()
df_merged['genres'] = df_merged['genres'].apply(ast.literal_eval)
df_merged = df_merged.explode('genres').reset_index(drop=True)
max_hours_genre = df_merged.loc[df_merged.groupby(['year', 'genres'])['hours_game'].idxmax()]
max_hours_genre = max_hours_genre.sort_values(by=['genres', 'hours_game', 'year'], ascending=[True, False, True])
max_hours_genre.to_csv('PlayTimeGenre.csv', index=False)

In [4]:
df_game = df_steam_games.drop(['name', 'developer', 'price', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(df_user_items,df_game,left_on='item_id', right_on='item_id', how='right')
merged_data.drop(['item_id'], axis=1, inplace=True)
merged_data.dropna(inplace=True)
merged_data['hours_game'] = merged_data['hours_game'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)
merged_data = merged_data.explode('genres').reset_index(drop=True)
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['hours_game'].idxmax()]
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'hours_game']]
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['hours_game'].sum().reset_index()
max_hours.to_csv('UserForGenre.csv', index=False)

In [5]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]
result = result.rename(columns={'year_x': 'year'})
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])
top3_by_year = grouped_result.groupby('year').head(3)
top3_by_year.to_csv('UsersRecommend.csv', index=False)

In [6]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
df_filtered = df_filtered.rename(columns={'year_x': 'year'})
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    
result_df = grouped_result.groupby('year').head(3)    
result_df.to_csv('UsersWorstDeveloper.csv', index=False)

In [7]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivotear df
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']
result_df = result_df.fillna(0)
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)
result_df.to_csv('sentiment_analysis.csv', index=False)